In [1]:
# This notebook uses the same Docker image as the Santa metric

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/santa-2024/sample_submission.csv


In [3]:
import pandas as pd
df=pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')


In [10]:
sets=[]

for i in range(6):
    sets.append(set(df.loc[i,'text'].split()))

In [5]:
for i in range(5)
   if sets[i].issubset(sets[5]):
       print(f'set {i} is subset of set5')

In [12]:
for i in range(1,6):
    if sets[0].issubset(sets[i]):
      print(f'set 0 is subset of set {i}')

set 0 is subset of set 1
set 0 is subset of set 5


In [14]:
for i in range(6):
    if i!=1 and sets[1].issubset(sets[i]):
      print(f'set 1 is subset of set {i}')

set 1 is subset of set 5


In [13]:
for i in range(6):
    if i!=2 and sets[2].issubset(sets[i]):
      print(f'set 2 is subset of set {i}')

set 2 is subset of set 3
set 2 is subset of set 5


In [15]:
for i in range(6):
    if i!=3 and sets[3].issubset(sets[i]):
      print(f'set 3 is subset of set {i}')

set 3 is subset of set 5


In [17]:
sets[4].issubset(sets[5])

True

In [19]:
lensets=[len(sets[i]) for i in range(6)]
print(f'len of sets {lensets}')

len of sets [10, 20, 20, 29, 50, 89]


In [20]:
lenlists = [len(df.loc[i,'text'].split()) for i in range(6)]
print(f'len of lists {lenlists}')

len of lists [10, 20, 20, 30, 50, 100]


In [23]:
temp=[]
for w in df.loc[3,'text'].split():
    if w in temp:
        print(f'the dup word in set 3 is {w}')
    else:
        temp.append(w)

the dup word in set 3 is cheer


In [25]:
temp=[]
for w in df.loc[5,'text'].split():
    if w in temp:
        print(f'the dup word in set 5 is {w}')
    else:
        temp.append(w)
print(f'len of temp {len(temp)}')

the dup word in set 5 is chimney
the dup word in set 5 is ornament
the dup word in set 5 is cheer
the dup word in set 5 is and
the dup word in set 5 is the
the dup word in set 5 is workshop
the dup word in set 5 is fireplace
the dup word in set 5 is night
the dup word in set 5 is the
the dup word in set 5 is of
the dup word in set 5 is and
len of temp 89
